***
## Настройка проекта для работы с картинками

Для подключения картинок необходимо выполнить несколько операций:

1. Добавить в модель `Birthday` поле для картинки.

2. Установить библиотеку для работы с изображениями.

3. Указать место для хранения загруженных изображений.

4. Выполнить миграции.

5. Изменить способ отправки данных из HTML-формы (сейчас форма умеет отправлять только текстовые данные; отправка файлов требует других настроек).

6. Доработать view-функцию для работы с файлами.

7. Настроить «раздачу» картинок сервером разработки Django.

***
## Новое поле в модели

В моделях Django есть специальный тип поля, предназначенный для изображений:  `models.ImageField()`. 

```py
# birthday/models.py
...

class Birthday(models.Model):
    first_name = models.CharField('Имя', max_length=20)
    last_name = models.CharField(
        'Фамилия', max_length=20, help_text='Опциональное поле', blank=True
    )
    birthday = models.DateField('Дата рождения', validators=(real_age,))
    image = models.ImageField('Фото', blank=True)

...
```

Если в этот момент запустить проект, появится ошибка:
```
ERRORS:
birthday.Birthday.image: (fields.E210) Cannot use ImageField because Pillow is not installed.
HINT: Get Pillow at https://pypi.org/project/Pillow/ or run command "python -m pip install Pillow". 
```

«Невозможно использовать класс ImageField без [библиотеки Pillow](https://pillow.readthedocs.io/en/latest/index.html)».

Библиотека Pillow отвечает за обработку графики; установите её в виртуальное окружение:

```bash
pip install Pillow==10.4.0 
```

***
## Директории для хранения загруженных файлов

Поле `image` настроено:

```py
...
image = models.ImageField('Фото', blank=True)
... 
```

Но если оставить описание поля в таком виде — загруженные картинки будут сохраняться прямо в корень проекта (рядом с файлом manage.py). Чтобы этого избежать — в настройках проекта нужно указать директорию, где будут храниться загруженные файлы. 

Общая директория для пользовательских файлов задаётся в настройках проекта, в константе `MEDIA_ROOT`. Словом *media* обычно определяют именно файлы, загруженные пользователями, и это слово используют для названия директории; впрочем, название может быть любым другим. 

Мы возьмём стандартный вариант названия:

```py
# acme_project/settings.py
...

MEDIA_ROOT = BASE_DIR / 'media' 
```

Директория для загрузки файлов из конкретного поля задаётся в классе ImageField в аргументе upload_to. Директория с таким названием будет создана в папке, указанной в настройках MEDIA_ROOT. 

Директорию для файлов из поля `Birthday.image` назовём birthdays_images:

```py
# birthday/models.py
...
image = models.ImageField('Фото', upload_to='birthdays_images', blank=True)
... 
```

По умолчанию данные из формы передаются в текстовом формате `ключ=значение`, но файлы — это не текст, а бинарные данные. Чтобы передавать такие данные из веб-формы — нужно изменить её настройки.

***
## Форматы данных, передаваемых из веб-формы

Формат данных, которые требуется отправить из веб-формы, указывается в HTML-теге `<form>` в атрибуте `enctype`. 

Таких форматов всего три; посмотрим, для чего они предназначены. Если потребуется более подробное описание — можно посмотреть [в документации для разработчиков](https://developer.mozilla.org/ru/docs/Web/HTML/Reference/Elements/form#attr-enctype) от компании Mozilla.

По умолчанию (если атрибут `enctype` не указан) форма отправляет данные в формате «ключ=значение»; в явном виде этот формат указывается так:

```html
<form enctype="application/x-www-form-urlencoded"> 
```

Слово `urlencoded` (URL кодирование) указывает, что все передаваемые символы (кроме [символов ASCII](https://www.w3schools.com/charsets/ref_html_ascii.asp) — латиницы и нескольких спецсимволов) кодируются специальным образом. Такое кодирование позволяет передавать через формы любые текстовые символы.

Другой вариант передачи данных из формы — `enctype="text/plain"` — простой текст без URL-кодирования, он в основном используется для целей отладки. 

Для передачи файлов применяется формат `enctype="multipart/form-data"`. Он-то нам и нужен.

Допишите в шаблоне *birthday/birthday.html* атрибут `enctype` к тегу `<form>`:

```html
<!-- templates/birthday/birthday.html -->
...
<form method="post" enctype="multipart/form-data">
... 
```

Теперь файл из формы будет отправляться, но на сервере он сохранён не будет: view-функция не готова обработать полученные файлы. 

Полученные в запросе файлы передаются не через `request.POST`, а через словарь `request.FILES`, и поэтому сейчас полученные файлы не передаются в объект формы и, следовательно, не сохраняются. 

Нужно поменять инициализацию формы и передать файлы в аргумент `files`:

```py
# birthday/views.py
...

def birthday(request, pk=None):
    if pk is not None:
        instance = get_object_or_404(Birthday, pk=pk)
    else:
        instance = None
    form = BirthdayForm(
        request.POST or None,
        # Файлы, переданные в запросе, указываются отдельно.
        files=request.FILES or None,
        instance=instance
    )
    ... 
```

И вот теперь Django сохранит файлы, отправленные через форму, на жёсткий диск, а в БД запишет путь до загруженного файла.

На странице редактирования записи отобразится ссылка на текущий файл и галочка для удаления файла.

На странице редактирования записи отобразится ссылка на текущий файл и галочка для удаления файла.

![alt text](https://pictures.s3.yandex.net/resources/180_1682599673.png)

Через форму редактирования можно прикрепить к записи другой файл, а можно просто отвязать файл от записи в БД. При замене или «отвязывании» файла от записи сами файлы не будут удалены и останутся лежать в той же директории, где и были. Для физического удаления таких файлов нужно использовать дополнительные библиотеки, например [django-cleanup](https://pypi.org/project/django-cleanup/).

***
## Раздача пользовательских картинок

Клик по названию файла в форме редактирования должен открывать картинку для просмотра, однако вместо этого при переходе по ссылке появится ошибка 404.

![alt text](https://pictures.s3.yandex.net/resources/181_1682599687.png)

Причина в том, что пересылка изображений пользователю не входит в основные задачи фреймворка Django. С одной стороны — это ресурсозатратный процесс, а с другой — процесс, предъявляющий повышенные требования к безопасности данных: вдруг злоумышленник захочет получить с сервера не только изображения, но и другие файлы, которые ему видеть не положено. 

Чтобы веб-сайт на Django мог отдавать реальным пользователям статические файлы — в их число входят и изображения, и файлы CSS, и файлы со скриптами на JavaScript, — используются другие программы, например, [nginx](https://nginx.org/ru/) — многофункциональная программа-сервер, обрабатывающая запросы и передающая их приложениям. При работе в связке с Django сервер nginx сам пересылает пользователю статические файлы, освобождая Django от рутинной работы.

Но пока nginx не подключён и не настроен — можно включить раздачу пользовательских файлов с помощью сервера разработки Django. Эта возможность предназначена исключительно для этапа разработки и не должна использоваться при работе с реальными пользователями. 

Режим раздачи статики сервером разработки настраивается в головном urls.py с помощью вспомогательной функции [django.conf.urls.static.static()](https://docs.djangoproject.com/en/3.2/ref/urls/#django.conf.urls.static.static):

```py
# acme_project/urls.py
# Импортируем настройки проекта.
from django.conf import settings
# Импортируем функцию, позволяющую серверу разработки отдавать файлы.
from django.conf.urls.static import static
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('', include('pages.urls')),
    path('admin/', admin.site.urls),
    path('birthday/', include('birthday.urls')),
    # В конце добавляем к списку вызов функции static.
] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT) 
```

Теперь картинка будет открываться по клику.

Этот способ будет работать только в режиме разработки (когда проект запускается командой `python manage.py runserver`) и при настройке DEBUG = True в `settings.py`. 

Когда проект запускается для работы с реальными пользователями, режим отладки `DEBUG = True` отключают, а запуск проекта выполняют иначе, не через `runserver`.

В режиме `DEBUG = False` Django перестаёт раздавать статические файлы: предполагается, что этим занимаются другие программы; если этих программ нет — статика не раздаётся вообще.

***
## Префикс в адресах статики

В вызове вспомогательной функции `static()` первым позиционным параметром идёт константа `settings.MEDIA_URL`. Она отвечает за то, чтобы перед адресами всех пользовательских файлов стоял определённый префикс. 

По умолчанию эта константа пуста, но довольно часто в проектах можно увидеть, что она переопределена. Например, можно добавить в настройки проекта константу `MEDIA_URL = 'media/'`

Теперь файлы будут доступны по другой ссылке:

```py
# Было:
http://127.0.0.1:8000/birthdays_images/image_2.png
# Стало:
http://127.0.0.1:8000/media/birthdays_images/image_2.png 
```

Использование такого префикса может быть требованием технического задания или же требованием сервера, на котором сайт запущен. Мы добавлять эту константу не будем, чтобы не усложнять себе жизнь.

***
## Показываем картинки

Изменим шаблон *birthday/birthday_list.html*:

* добавим `<div>`, в который будет выводиться картинка;

* установим проверку: выводить тег `<img>` только в том случае, если картинка есть в объекте;

* вспомним работу с шаблонами: в теге `<img>` укажем адрес картинки через атрибут `url` объекта `image`;

* добавим [классы фреймворка Bootstrap](https://getbootstrap.com/docs/5.1/layout/grid/#grid-options):

    `row` — строка, объединяющая все элементы отдельной записи: картинку, текст и ссылки;

    `col-2` и `col-10` — две колонки, вложенные в строку.

    В блоке `col-2` — картинка, в блоке `col-10` — содержимое записи и ссылки.

```py
{% extends "base.html" %}

{% block content %}
  {% for birthday in birthdays %}
    <!-- Добавим класс row: каждая запись — это строка -->
    <div class="row">
      <!-- Первая "колонка" в строке, её ширина — 2/12 -->
      <div class="col-2">  
        {% if birthday.image %}
          <!-- У объекта картинки image есть атрибут url — его и укажем в src -->
          <img src="{{ birthday.image.url }}" height="100">
        {% endif %}
      </div>

      <!-- Вторая "колонка" в строке, её ширина — 10/12 -->
      <div class="col-10">  
        <div>
          {{ birthday.first_name }} {{ birthday.last_name }} — {{ birthday.birthday }}
        </div>      
        <div>
          <a href="{% url 'birthday:edit' birthday.id %}">Изменить запись</a> | <a href="{% url 'birthday:delete' birthday.id %}">Удалить запись</a>
        </div>
      </div>
      {% if not forloop.last %}
        <!-- Класс mt-3 устанавливает отступ сверху (margin-top) в 30 px -->
        <hr class="mt-3"> 
      {% endif %}
    </div>
  {% endfor %}
{% endblock %} 
```

> Работа с формами — непростая задача, особенно в первое время. Чтобы было проще, используйте [шпаргалку](https://code.s3.yandex.net/Python-dev/cheatsheets/032-django-formy-shpora/032-django-formy-shpora.html). Сохраните её в закладки и заглядывайте, если столкнётесь с трудностями.